In [1]:
# ============================================================================
# COMPREHENSIVE DEBUGGING CODE FOR method_runner
# Run this in a Jupyter notebook cell in the notebooks/ folder
# Copy-paste the entire code block below
# ============================================================================

from pathlib import Path
import sys
import json
import numpy as np

# Setup path
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.methods.method_runner import run_talent_method, get_available_methods

print("=" * 80)
print("DEBUGGING: TALENT method_runner Analysis")
print("=" * 80)

# ============================================================================
# TEST 1: Single fold with HPO - inspect everything
# ============================================================================
print("\nTEST 1: Single fold MLP + HPO (inspect results dict)")
print("-" * 80)

results = run_talent_method(
    task="pd",
    dataset="0009.german",
    test_size=0.2,
    val_size=0.2,
    cv_splits=1,
    seed=42,
    method="mlp",
    max_epoch=10,
    tune=True,
    n_trials=3,
    verbose=True,
)

print("\n" + "=" * 80)
print("RESULTS INSPECTION - FOLD 0")
print("=" * 80)

fold = results[0]

print("\n1. RESULT DICT KEYS:")
for key in sorted(fold.keys()):
    print(f"   - {key}: {type(fold[key])}")

print("\n2. PREDICTIONS:")
print(f"   y_true shape: {fold['y_true'].shape}")
print(f"   y_true dtype: {fold['y_true'].dtype}")
print(f"   y_true values: {np.unique(fold['y_true'])}")
print(f"   y_true first 10: {fold['y_true'][:10]}")

print(f"\n   y_pred shape: {fold['y_pred'].shape}")
print(f"   y_pred dtype: {fold['y_pred'].dtype}")
if fold['y_pred'].ndim == 1:
    print(f"   y_pred is 1D: {fold['y_pred'][:10]}")
else:
    print(f"   y_pred is {fold['y_pred'].ndim}D: {fold['y_pred'][:3]}")

print("\n3. METRICS:")
print(f"   Primary metric: {fold['primary_metric']}")
print(f"   Metric names: {fold['metric_names']}")
print(f"   Metrics: {fold['metrics']}")
if isinstance(fold['metrics'], (list, tuple)):
    for name, val in zip(fold['metric_names'], fold['metrics']):
        print(f"      {name}: {val}")

print("\n4. TRAINING INFO:")
print(f"   Training time: {fold['train_time']:.2f}s")
print(f"   Validation loss: {fold['val_loss']}")
print(f"   Method: {fold['method']}")
print(f"   Dataset: {fold['dataset']}")

print("\n5. HPO CONFIG:")
hpo_config = fold['optimal_hpo_config']
if hpo_config:
    print(f"   HPO config found: YES")
    print(f"   Config structure: {json.dumps(hpo_config, indent=2)[:1000]}...")
else:
    print(f"   HPO config found: NO")

# ============================================================================
# TEST 2: 3 folds with HPO - check config reuse
# ============================================================================
print("\n" + "=" * 80)
print("TEST 2: 3 folds with HPO - check config reuse across folds")
print("=" * 80)

results_3fold = run_talent_method(
    task="pd",
    dataset="0009.german",
    test_size=0.2,
    val_size=0.2,
    cv_splits=3,
    seed=42,
    method="mlp",
    max_epoch=10,
    tune=True,
    n_trials=3,
    verbose=False,
)

print("\nCROSS-FOLD ANALYSIS:")
configs = []
metrics_list = []

for fold_id, fold_result in results_3fold.items():
    hpo_config = fold_result['optimal_hpo_config']
    metric_names = fold_result['metric_names']
    metrics = fold_result['metrics']
    
    configs.append(hpo_config)
    metrics_list.append(dict(zip(metric_names, metrics)) if metric_names else {})
    
    print(f"\n   Fold {fold_id}:")
    print(f"      Metrics: {metrics_list[-1]}")
    if hpo_config and 'mlp' in hpo_config:
        if 'fit' in hpo_config['mlp']:
            lr = hpo_config['mlp']['fit'].get('lr', 'N/A')
            print(f"      Tuned LR: {lr}")

print("\n   CONFIG REUSE:")
config_0 = configs[0]
for fold_id in range(1, len(configs)):
    is_same = configs[fold_id] == config_0
    print(f"      Fold {fold_id} == Fold 0 config: {is_same}")

# ============================================================================
# TEST 3: Compare WITH HPO vs WITHOUT HPO
# ============================================================================
print("\n" + "=" * 80)
print("TEST 3: WITH HPO vs WITHOUT HPO comparison")
print("=" * 80)

print("\n   Running WITHOUT HPO...")
results_no_hpo = run_talent_method(
    task="pd", dataset="0009.german", test_size=0.2, val_size=0.2,
    cv_splits=1, seed=42, method="mlp", max_epoch=10,
    tune=False, verbose=False,
)

print("   Running WITH HPO...")
results_with_hpo = run_talent_method(
    task="pd", dataset="0009.german", test_size=0.2, val_size=0.2,
    cv_splits=1, seed=42, method="mlp", max_epoch=10,
    tune=True, n_trials=3, verbose=False,
)

fold_no_hpo = results_no_hpo[0]
fold_with_hpo = results_with_hpo[0]

print("\n   COMPARISON:")
print(f"   Training time WITHOUT HPO: {fold_no_hpo['train_time']:.2f}s")
print(f"   Training time WITH HPO:    {fold_with_hpo['train_time']:.2f}s")

metrics_no_hpo = dict(zip(fold_no_hpo['metric_names'], fold_no_hpo['metrics']))
metrics_with_hpo = dict(zip(fold_with_hpo['metric_names'], fold_with_hpo['metrics']))

print(f"\n   WITHOUT HPO metrics: {metrics_no_hpo}")
print(f"   WITH HPO metrics:    {metrics_with_hpo}")

print(f"\n   Improvement:")
for metric_name in fold_no_hpo['metric_names']:
    val_no_hpo = metrics_no_hpo[metric_name]
    val_with_hpo = metrics_with_hpo[metric_name]
    improvement = val_with_hpo - val_no_hpo
    print(f"      {metric_name}: {val_no_hpo:.4f} → {val_with_hpo:.4f} ({improvement:+.4f})")

print("\n" + "=" * 80)
print("DEBUG COMPLETE")
print("=" * 80)

DEBUGGING: TALENT method_runner Analysis

TEST 1: Single fold MLP + HPO (inspect results dict)
--------------------------------------------------------------------------------

Running mlp (deep) on 0009.german (PD)
using gpu: 0
{'batch_size': 1024,
 'cat_min_frequency': 0.0,
 'cat_nan_policy': 'new',
 'cat_policy': 'ordinal',
 'config': {'general': {},
            'model': {'d_layers': [384, 384], 'dropout': 0.1},
            'training': {'lr': 0.0003, 'weight_decay': 1e-05}},
 'dataset': '0009.german',
 'dataset_path': './data',
 'evaluate_option': 'best-val',
 'gpu': '0',
 'max_epoch': 200,
 'model_path': './models',
 'model_type': 'mlp',
 'n_bins': 2,
 'n_trials': 3,
 'normalization': 'standard',
 'num_nan_policy': 'mean',
 'num_policy': 'none',
 'retune': False,
 'save_path': './models\\0009.german-mlp-Tune\\Epoch200BZ1024-Norm-standard-Nan-mean-new-Cat-ordinal',
 'seed': 0,
 'seed_num': 42,
 'tune': True,
 'use_float': False,
 'workers': 0}

Preprocessing configuration:
  - cat_p

1it [00:00, ?it/s]


epoch 0, val, loss=0.6609 classification result=0.7063
Epoch: 0, Time cost: 0.017267465591430664
epoch 1, train 1/1, loss=0.6561 lr=0.0003
best epoch 0, best val res=0.7063


1it [00:00, 84.42it/s]


epoch 1, val, loss=0.6394 classification result=0.7000
Epoch: 1, Time cost: 0.09967231750488281
epoch 2, train 1/1, loss=0.6338 lr=0.0003
best epoch 0, best val res=0.7063


...

epoch 9, val, loss=0.5821 classification result=0.7000


Epoch: 9, Time cost: 0.024724960327148438
best epoch 0, best val res=0.7063


1it [00:00, 328.50it/s]

Test: loss=0.6564
[Accuracy]=0.7050
[Avg_Recall]=0.5083
[Avg_Precision]=0.8518
[F1]=0.0328
[LogLoss]=0.6564
[AUC]=0.4946

Fold 1 results:
  Accuracy: 0.7050
  Avg_Recall: 0.5083
  Avg_Precision: 0.8518
  F1: 0.0328
  LogLoss: 0.6564
  AUC: 0.4946

Completed 1 folds for mlp


RESULTS INSPECTION - FOLD 0


KeyError: 0

In [ ]:
"""
Comprehensive Test Suite for method_runner.py with HPO
======================================================

This notebook tests the method_runner with:
- Multiple datasets
- Multiple methods (with and without HPO support)
- HPO enabled and disabled
- Extensive debugging output
"""

import sys
import time
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
from pprint import pprint

# Ensure we can import from the src folder (project root = parent of notebooks/)
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.methods.method_runner import run_talent_method, get_available_methods

warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION
# =============================================================================

# Test configuration
TEST_CONFIG = {
    "row_limit": 5000,           # Limit rows for faster testing
    "cv_splits": 3,              # Number of CV folds
    "test_size": 0.2,            # Test set size
    "val_size": 0.2,             # Validation set size
    "seed": 42,                  # Random seed
    "n_trials_hpo": 10,          # HPO trials (reduced for testing)
}

# Datasets to test
DATASETS = [
    "0001.gmsc",     # German Credit
    "0002.aust",     # Australian Credit
    "0003.japa",     # Japanese Credit
]

# Methods to test
METHODS_TO_TEST = {
    # Classical methods (support HPO)
    "classical_with_hpo": [
        "xgboost",
        "lightgbm",
        "RandomForest",
    ],
    
    # Deep methods (support HPO)  
    "deep_with_hpo": [
        "mlp",
        "resnet",
    ],
    
    # Zero-shot methods (NO HPO support)
    "zero_shot_no_hpo": [
        "tabpfn",
    ]
}

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def print_section(title, char="=", width=80):
    """Print a formatted section header."""
    print(f"\n{char * width}")
    print(f"{title.center(width)}")
    print(f"{char * width}\n")


def print_subsection(title, char="-", width=80):
    """Print a formatted subsection header."""
    print(f"\n{char * width}")
    print(f" {title}")
    print(f"{char * width}")


def print_dict(d, indent=0):
    """Pretty print a dictionary."""
    for key, value in d.items():
        print("  " * indent + f"{key}:", end=" ")
        if isinstance(value, dict):
            print()
            print_dict(value, indent + 1)
        elif isinstance(value, (list, tuple)) and len(str(value)) > 60:
            print(f"{type(value).__name__} with {len(value)} items")
        else:
            print(value)


def format_time(seconds):
    """Format time in seconds to human-readable string."""
    if seconds < 60:
        return f"{seconds:.2f}s"
    elif seconds < 3600:
        return f"{seconds/60:.2f}m"
    else:
        return f"{seconds/3600:.2f}h"


def compute_metrics_summary(results):
    """Compute summary statistics across folds."""
    train_times = [r['train_time'] for r in results.values()]
    
    # Extract metrics if they exist
    metrics_dict = {}
    for fold_id, fold_results in results.items():
        if isinstance(fold_results.get('metrics'), (tuple, list)):
            for i, metric in enumerate(fold_results['metrics']):
                if f'metric_{i}' not in metrics_dict:
                    metrics_dict[f'metric_{i}'] = []
                metrics_dict[f'metric_{i}'].append(metric)
    
    summary = {
        'n_folds': len(results),
        'train_time_mean': np.mean(train_times),
        'train_time_std': np.std(train_times),
        'train_time_total': np.sum(train_times),
    }
    
    # Add metric summaries
    for metric_name, values in metrics_dict.items():
        summary[f'{metric_name}_mean'] = np.mean(values)
        summary[f'{metric_name}_std'] = np.std(values)
    
    return summary


# =============================================================================
# TEST FUNCTION
# =============================================================================

def test_method_on_dataset(
    method: str,
    dataset: str,
    task: str = "pd",
    tune: bool = False,
    verbose: bool = False,
):
    """
    Test a single method on a single dataset.
    
    Args:
        method: Method name (TALENT canonical)
        dataset: Dataset name
        task: Task type ('pd' or 'lgd')
        tune: Whether to enable HPO
        verbose: Whether to print verbose output
        
    Returns:
        Dictionary with test results and timing info
    """
    print_subsection(f"Testing: {method} on {dataset} {'WITH' if tune else 'WITHOUT'} HPO")
    
    test_result = {
        'method': method,
        'dataset': dataset,
        'task': task,
        'tune': tune,
        'success': False,
        'error': None,
        'results': None,
        'timing': {},
        'summary': {},
    }
    
    try:
        # Record start time
        start_time = time.time()
        
        print(f"⏳ Starting test...")
        print(f"   Method: {method}")
        print(f"   Dataset: {dataset}")
        print(f"   Task: {task.upper()}")
        print(f"   HPO: {'Enabled' if tune else 'Disabled'}")
        print(f"   Row limit: {TEST_CONFIG['row_limit']}")
        print(f"   CV splits: {TEST_CONFIG['cv_splits']}")
        
        # Run the method
        results = run_talent_method(
            task=task,
            dataset=dataset,
            test_size=TEST_CONFIG['test_size'],
            val_size=TEST_CONFIG['val_size'],
            cv_splits=TEST_CONFIG['cv_splits'],
            seed=TEST_CONFIG['seed'],
            row_limit=TEST_CONFIG['row_limit'],
            method=method,
            tune=tune,
            n_trials=TEST_CONFIG['n_trials_hpo'] if tune else 0,
            verbose=verbose,
        )
        
        # Record end time
        end_time = time.time()
        total_time = end_time - start_time
        
        # Compute summary
        summary = compute_metrics_summary(results)
        
        # Store results
        test_result['success'] = True
        test_result['results'] = results
        test_result['timing'] = {
            'total_time': total_time,
            'per_fold_avg': total_time / len(results),
        }
        test_result['summary'] = summary
        
        # Print success info
        print(f"\n✅ SUCCESS!")
        print(f"   Total time: {format_time(total_time)}")
        print(f"   Avg time per fold: {format_time(total_time / len(results))}")
        print(f"   Number of folds: {len(results)}")
        
        if summary:
            print(f"\n📊 Performance Summary:")
            print(f"   Training time: {summary['train_time_mean']:.2f}s ± {summary['train_time_std']:.2f}s")
            
            # Print first few metrics
            metric_keys = [k for k in summary.keys() if k.startswith('metric_') and k.endswith('_mean')]
            for i, metric_key in enumerate(metric_keys[:3]):  # First 3 metrics
                mean_val = summary[metric_key]
                std_key = metric_key.replace('_mean', '_std')
                std_val = summary[std_key]
                print(f"   Metric {i}: {mean_val:.4f} ± {std_val:.4f}")
        
    except Exception as e:
        # Record error
        test_result['success'] = False
        test_result['error'] = str(e)
        
        print(f"\n❌ FAILED!")
        print(f"   Error: {e}")
        print(f"   Error type: {type(e).__name__}")
        
        # Print traceback for debugging
        import traceback
        print(f"\n🐛 Traceback:")
        traceback.print_exc()
    
    return test_result


# =============================================================================
# MAIN TEST SUITE
# =============================================================================

def run_test_suite():
    """Run the complete test suite."""
    
    print_section("METHOD RUNNER TEST SUITE - WITH HPO SUPPORT", "=", 80)
    
    # Print configuration
    print("📋 Test Configuration:")
    print_dict(TEST_CONFIG, indent=1)
    
    print(f"\n📂 Datasets to test: {len(DATASETS)}")
    for dataset in DATASETS:
        print(f"   - {dataset}")
    
    print(f"\n🔧 Methods to test:")
    total_methods = sum(len(methods) for methods in METHODS_TO_TEST.values())
    print(f"   Total: {total_methods} methods")
    for category, methods in METHODS_TO_TEST.items():
        print(f"   - {category}: {', '.join(methods)}")
    
    # Storage for all results
    all_results = []
    
    # ==========================================================================
    # TEST 1: Classical Methods WITH HPO
    # ==========================================================================
    
    print_section("TEST 1: CLASSICAL METHODS WITH HPO", "=")
    
    for method in METHODS_TO_TEST['classical_with_hpo']:
        for dataset in DATASETS[:1]:  # Test on first dataset only for speed
            result = test_method_on_dataset(
                method=method,
                dataset=dataset,
                task="pd",
                tune=True,  # Enable HPO
                verbose=False,
            )
            all_results.append(result)
    
    # ==========================================================================
    # TEST 2: Classical Methods WITHOUT HPO (for comparison)
    # ==========================================================================
    
    print_section("TEST 2: CLASSICAL METHODS WITHOUT HPO (Baseline)", "=")
    
    for method in METHODS_TO_TEST['classical_with_hpo'][:1]:  # Just test one
        for dataset in DATASETS[:1]:
            result = test_method_on_dataset(
                method=method,
                dataset=dataset,
                task="pd",
                tune=False,  # Disable HPO
                verbose=False,
            )
            all_results.append(result)
    
    # ==========================================================================
    # TEST 3: Deep Methods WITH HPO
    # ==========================================================================
    
    print_section("TEST 3: DEEP LEARNING METHODS WITH HPO", "=")
    
    for method in METHODS_TO_TEST['deep_with_hpo']:
        for dataset in DATASETS[:1]:  # Test on first dataset only
            result = test_method_on_dataset(
                method=method,
                dataset=dataset,
                task="pd",
                tune=True,  # Enable HPO
                verbose=False,
            )
            all_results.append(result)
    
    # ==========================================================================
    # TEST 4: Zero-Shot Methods (Should NOT use HPO)
    # ==========================================================================
    
    print_section("TEST 4: ZERO-SHOT METHODS (NO HPO)", "=")
    
    for method in METHODS_TO_TEST['zero_shot_no_hpo']:
        for dataset in DATASETS[:1]:
            result = test_method_on_dataset(
                method=method,
                dataset=dataset,
                task="pd",
                tune=False,  # MUST be False for TabPFN
                verbose=False,
            )
            all_results.append(result)
    
    # ==========================================================================
    # TEST 5: Error Case - Try HPO with TabPFN (Should Fail)
    # ==========================================================================
    
    print_section("TEST 5: ERROR HANDLING - HPO with TabPFN (Expected to Fail)", "=")
    
    print("⚠️  This test is EXPECTED to fail (TabPFN doesn't support HPO)")
    print("   We're testing that the error handling works correctly.\n")
    
    result = test_method_on_dataset(
        method="tabpfn",
        dataset=DATASETS[0],
        task="pd",
        tune=True,  # This should cause an error
        verbose=False,
    )
    all_results.append(result)
    
    if not result['success']:
        print("\n✅ Error handling worked correctly!")
        print(f"   Expected error occurred: {result['error']}")
    
    # ==========================================================================
    # FINAL SUMMARY
    # ==========================================================================
    
    print_section("FINAL SUMMARY", "=")
    
    # Create summary dataframe
    summary_data = []
    for result in all_results:
        row = {
            'method': result['method'],
            'dataset': result['dataset'],
            'hpo': 'Yes' if result['tune'] else 'No',
            'success': '✓' if result['success'] else '✗',
            'total_time': result['timing'].get('total_time', np.nan),
            'error': result['error'] if result['error'] else '',
        }
        
        # Add performance metrics if available
        if result['summary']:
            row['train_time_mean'] = result['summary'].get('train_time_mean', np.nan)
            row['metric_0_mean'] = result['summary'].get('metric_0_mean', np.nan)
            row['metric_5_mean'] = result['summary'].get('metric_5_mean', np.nan)  # Usually AUC
        
        summary_data.append(row)
    
    df_summary = pd.DataFrame(summary_data)
    
    print("\n📊 Test Results Summary:")
    print(df_summary.to_string(index=False))
    
    # Success rate
    n_success = sum(1 for r in all_results if r['success'])
    n_total = len(all_results)
    success_rate = n_success / n_total * 100
    
    print(f"\n📈 Overall Success Rate: {n_success}/{n_total} ({success_rate:.1f}%)")
    
    # HPO comparison
    print("\n🔍 HPO Impact Analysis:")
    
    hpo_results = [r for r in all_results if r['tune'] and r['success']]
    no_hpo_results = [r for r in all_results if not r['tune'] and r['success']]
    
    if hpo_results and no_hpo_results:
        hpo_times = [r['timing']['total_time'] for r in hpo_results]
        no_hpo_times = [r['timing']['total_time'] for r in no_hpo_results]
        
        print(f"   WITH HPO:")
        print(f"      Avg time: {np.mean(hpo_times):.2f}s")
        print(f"      Tests: {len(hpo_results)}")
        
        print(f"   WITHOUT HPO:")
        print(f"      Avg time: {np.mean(no_hpo_times):.2f}s")
        print(f"      Tests: {len(no_hpo_results)}")
        
        speedup = np.mean(hpo_times) / np.mean(no_hpo_times)
        print(f"   Time ratio: {speedup:.1f}x slower with HPO")
    
    # Method comparison
    print("\n🏆 Method Performance Ranking (by metric_0):")
    
    successful_results = [r for r in all_results if r['success'] and r['summary']]
    if successful_results:
        method_perf = []
        for result in successful_results:
            if 'metric_0_mean' in result['summary']:
                method_perf.append({
                    'method': result['method'],
                    'hpo': 'Yes' if result['tune'] else 'No',
                    'metric_0': result['summary']['metric_0_mean'],
                })
        
        if method_perf:
            df_perf = pd.DataFrame(method_perf).sort_values('metric_0', ascending=False)
            print(df_perf.to_string(index=False))
    
    print_section("TEST SUITE COMPLETE!", "=")
    
    return all_results, df_summary


# =============================================================================
# DEBUGGING HELPER
# =============================================================================

def debug_single_run(method="mlp", dataset="0001.gmsc", tune=True):
    """
    Run a single method with maximum verbosity for debugging.
    
    Args:
        method: Method to test
        dataset: Dataset to use
        tune: Enable HPO
    """
    print_section(f"DEBUG RUN: {method} on {dataset}", "=")
    
    print("🔍 Running with MAXIMUM verbosity for debugging...\n")
    
    # Import method_runner internals for inspection
    from src.methods.method_runner import (
        get_available_methods,
        validate_method,
        DEEP_METHODS,
        CLASSICAL_METHODS
    )
    
    # Validate method
    print("1️⃣ VALIDATING METHOD")
    try:
        validated_method, is_deep = validate_method(method)
        print(f"   ✓ Method '{method}' is valid")
        print(f"   ✓ Type: {'Deep Learning' if is_deep else 'Classical'}")
        print(f"   ✓ Supports HPO: {'Yes' if method not in ['tabpfn', 'PFN-v2', 'tabptm'] else 'No'}")
    except Exception as e:
        print(f"   ✗ Validation failed: {e}")
        return
    
    # Check dataset
    print(f"\n2️⃣ CHECKING DATASET")
    print(f"   Dataset: {dataset}")
    print(f"   Row limit: {TEST_CONFIG['row_limit']}")
    
    # Run with verbose output
    print(f"\n3️⃣ RUNNING METHOD")
    print(f"   HPO: {'ENABLED' if tune else 'DISABLED'}")
    if tune:
        print(f"   Number of trials: {TEST_CONFIG['n_trials_hpo']}")
    
    start_time = time.time()
    
    try:
        results = run_talent_method(
            task="pd",
            dataset=dataset,
            test_size=0.2,
            val_size=0.2,
            cv_splits=3,
            seed=42,
            row_limit=TEST_CONFIG['row_limit'],
            method=method,
            tune=tune,
            n_trials=TEST_CONFIG['n_trials_hpo'],
            verbose=True,  # MAXIMUM VERBOSITY
        )
        
        elapsed = time.time() - start_time
        
        print(f"\n4️⃣ RESULTS")
        print(f"   ✓ Completed successfully")
        print(f"   ✓ Time: {format_time(elapsed)}")
        print(f"   ✓ Number of folds: {len(results)}")
        
        # Inspect first fold in detail
        print(f"\n5️⃣ DETAILED INSPECTION OF FOLD 1")
        fold_1 = results[1]
        
        print(f"   Keys: {list(fold_1.keys())}")
        print(f"   Method: {fold_1['method']}")
        print(f"   Dataset: {fold_1['dataset']}")
        print(f"   Task: {fold_1['task']}")
        print(f"   Training time: {fold_1['train_time']:.2f}s")
        print(f"   y_true shape: {fold_1['y_true'].shape}")
        print(f"   y_pred shape: {fold_1['y_pred'].shape}")
        print(f"   Metrics: {fold_1['metrics']}")
        
        return results
        
    except Exception as e:
        print(f"\n   ✗ FAILED: {e}")
        import traceback
        traceback.print_exc()
        return None


# =============================================================================
# QUICK TEST FUNCTIONS
# =============================================================================

def quick_test_hpo():
    """Quick test to verify HPO is working."""
    print_section("QUICK HPO TEST", "=")
    
    print("Testing XGBoost with HPO on small dataset...\n")
    
    result = test_method_on_dataset(
        method="xgboost",
        dataset="0001.gmsc",
        task="pd",
        tune=True,
        verbose=False,
    )
    
    if result['success']:
        print("\n✅ HPO test PASSED!")
        print(f"   Check if HPO improved performance by comparing with baseline")
    else:
        print("\n❌ HPO test FAILED!")
        print(f"   Error: {result['error']}")
    
    return result


def quick_test_comparison():
    """Quick test comparing with and without HPO."""
    print_section("QUICK COMPARISON: HPO vs No HPO", "=")
    
    method = "xgboost"
    dataset = "0001.gmsc"
    
    print(f"Testing {method} on {dataset}...\n")
    
    # Without HPO
    print("1️⃣ WITHOUT HPO (baseline):")
    result_no_hpo = test_method_on_dataset(
        method=method,
        dataset=dataset,
        task="pd",
        tune=False,
        verbose=False,
    )
    
    # With HPO
    print("\n2️⃣ WITH HPO:")
    result_with_hpo = test_method_on_dataset(
        method=method,
        dataset=dataset,
        task="pd",
        tune=True,
        verbose=False,
    )
    
    # Compare
    print_section("COMPARISON RESULTS", "-")
    
    if result_no_hpo['success'] and result_with_hpo['success']:
        no_hpo_metric = result_no_hpo['summary'].get('metric_0_mean', 0)
        with_hpo_metric = result_with_hpo['summary'].get('metric_0_mean', 0)
        
        improvement = (with_hpo_metric - no_hpo_metric) / no_hpo_metric * 100
        
        print(f"Without HPO: metric = {no_hpo_metric:.4f}")
        print(f"With HPO:    metric = {with_hpo_metric:.4f}")
        print(f"Improvement: {improvement:+.2f}%")
        
        no_hpo_time = result_no_hpo['timing']['total_time']
        with_hpo_time = result_with_hpo['timing']['total_time']
        time_ratio = with_hpo_time / no_hpo_time
        
        print(f"\nWithout HPO: time = {format_time(no_hpo_time)}")
        print(f"With HPO:    time = {format_time(with_hpo_time)}")
        print(f"Time ratio:  {time_ratio:.1f}x slower")
    else:
        print("⚠️  Could not compare due to errors")
    
    return result_no_hpo, result_with_hpo


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("\n" + "="*80)
    print(" TALENT METHOD RUNNER - COMPREHENSIVE TEST SUITE ".center(80))
    print("="*80 + "\n")
    
    print("Select test mode:")
    print("  1. Full test suite (slow but comprehensive)")
    print("  2. Quick HPO test (fast)")
    print("  3. Quick comparison (HPO vs No HPO)")
    print("  4. Debug single run (maximum verbosity)")
    print("  5. Run all tests")
    
    # For notebook execution, uncomment one of these:
    
    # Option 1: Full test suite
    # all_results, df_summary = run_test_suite()
    
    # Option 2: Quick HPO test
    # result = quick_test_hpo()
    
    # Option 3: Quick comparison
    # result_no_hpo, result_with_hpo = quick_test_comparison()
    
    # Option 4: Debug run
    # results = debug_single_run(method="mlp", dataset="0001.gmsc", tune=True)
    
    # Option 5: Run all
    print("\n🚀 Running FULL TEST SUITE...\n")
    all_results, df_summary = run_test_suite()
    
    print("\n✅ All tests complete! Results stored in 'all_results' and 'df_summary'")


                TALENT METHOD RUNNER - COMPREHENSIVE TEST SUITE                 

Select test mode:
  1. Full test suite (slow but comprehensive)
  2. Quick HPO test (fast)
  3. Quick comparison (HPO vs No HPO)
  4. Debug single run (maximum verbosity)
  5. Run all tests

🚀 Running FULL TEST SUITE...


                  METHOD RUNNER TEST SUITE - WITH HPO SUPPORT                   

📋 Test Configuration:
  row_limit: 5000
  cv_splits: 3
  test_size: 0.2
  val_size: 0.2
  seed: 42
  n_trials_hpo: 10

📂 Datasets to test: 3
   - 0001.gmsc
   - 0002.aust
   - 0003.japa

🔧 Methods to test:
   Total: 6 methods
   - classical_with_hpo: xgboost, lightgbm, RandomForest
   - deep_with_hpo: mlp, resnet
   - zero_shot_no_hpo: tabpfn

                       TEST 1: CLASSICAL METHODS WITH HPO                       


--------------------------------------------------------------------------------
 Testing: xgboost on 0001.gmsc WITH HPO
---------------------------------------------------------------

In [ ]:
results = run_talent_method(
    task="pd", dataset="adult", test_size=0.2, val_size=0.2,
    cv_splits=1, seed=42, method="mlp", max_epoch=5, verbose=True
)
print(results)